In [8]:
#read yaml file
import yaml

with open("configs/cem_kp.yaml", 'r') as stream:
    try:
        config = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)
print(config)


{'solver': {'type': 'vqa', 'optimizer': {'type': 'scipy', 'maxfun': 200}, 'pqc': {'type': 'wfqaoa', 'layers': 5, 'penalty': 3}, 'params_inits': {'angles': [[0.5, 0.5, 0.5, 0.5, 0.5], [1, 1, 1, 1, 1]], 'hyper_args': [1, 2.5, 2.5]}}, 'problem': {'type': 'knapsack', 'max_weight': 2, 'items': [[1, 2], [1, 2], [1, 1]]}}


In [11]:
from QHyper.solvers import solver_from_config

solver = solver_from_config(config)


In [12]:
solver.solve()


SolverResult(results_probabilities={'00000': 0.1580399501811467, '00001': 0.010270379506994722, '00010': 0.009383422850867096, '00011': 0.024490163676211298, '00100': 0.010529240688646633, '00101': 0.02035298313640454, '00110': 0.30944026871447716, '00111': 0.005766319932680871, '01000': 0.006865542940697496, '01001': 0.028169813645767587, '01010': 0.053542323447889846, '01011': 0.0074258126091402985, '01100': 0.016559034157724518, '01101': 0.03728290079671686, '01110': 0.009516489563162749, '01111': 0.018076475550385693, '10000': 0.006865542940697455, '10001': 0.028169813645767497, '10010': 0.05354232344788981, '10011': 0.007425812609140283, '10100': 0.01655903415772452, '10101': 0.037282900796716895, '10110': 0.009516489563162766, '10111': 0.01807647555038569, '11000': 0.002069500791451921, '11001': 0.024916352780264647, '11010': 0.01616903424058875, '11011': 0.014199866153932665, '11100': 0.0145984985105855, '11101': 0.017839945779733742, '11110': 0.0028087376660474507, '11111': 0.0